In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import h5py
import os

from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import savgol_filter
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d

from utils import load_syllable, rle, get_pose, select_kp

In [3]:
def SelectPos(test_coor_path, syllable_path, select_syllable, syllable_name):
    # pass
    test_coor = get_pose(test_coor_path, concatenate=False)
    test_syllable = load_syllable(syllable_path)
    select_pos_list = select_kp(test_coor, test_syllable, select_syllable, syllable_name)
    
    return select_pos_list

In [4]:
# data path and selected syllables
test_coor_path = r"D:\ZhouLab\HJQ\mice_3d_pose\test_session_with_ephys_downsam\test_with_ephys_filter_downsam_5.npy"
syllable_path = r"D:\ZhouLab\HJQ\model_test_with_ephys_filter_downsam_5\20000.0\results.h5"
select_syllable = 3
syllable_name = 'diving'

dots_pos = SelectPos(test_coor_path, syllable_path, select_syllable, syllable_name)

In [5]:
dots_pos[5].shape

(24, 22, 3)

In [6]:
b2 = np.transpose(dots_pos[5], (0,2,1))

# Data processing
mujoco_timestep = 0.0025 # unit: s
fps = 50 # unit: Hz

# Define the time vector based on the original frame rate
t_original = np.arange(0, b2.shape[0] * (1 / fps), 1 / fps)

# Create a new time vector based on the desired timestep for MuJoCo
t_new = np.arange(0, b2.shape[0] * mujoco_timestep, mujoco_timestep)

# Initialize arrays to store interpolated data
b2_interpolated = np.zeros((len(t_new), b2.shape[1], b2.shape[2]))

# Interpolate each dimension separately
for i in range(b2.shape[1]):
    for j in range(b2.shape[2]):
        # Perform cubic spline interpolation
        cs = CubicSpline(t_original, b2[:, i, j])
        b2_interpolated[:, i, j] = cs(t_new)

b2 = b2_interpolated

In [7]:
# Initialize dictionaries for angles and skeleton components
Dofs = ['root_x', 'root_y', 'root_z', 
        'root_rot_x', 'root_rot_y', 'root_rot_z',
        'neck_x', 'neck_y', 'neck_z',
        'RScapula_r1', 'RScapula_r2', 'RScapula_r3', 'RScapula_r4',
        'RShoulder_flexion','RShoulder_adduction', 'RShoulder_rotation', 
        'RElbow_flexion',
        'RRadius_rotation', 'RWrist_adduction', 'RWrist_flexion', 
        'RClavicle_r1', 'RClavicle_r2',
        'LScapula_r1', 'LScapula_r2', 'LScapula_r3', 'LScapula_r4',
        'LShoulder_flexion','LShoulder_adduction', 'LShoulder_rotation', 
        'LElbow_flexion',
        'LRadius_rotation', 'LWrist_adduction', 'LWrist_flexion', 
        'LClavicle_r1', 'LClavicle_r2',
        'waist_x','waist_y','waist_z',
        'RHip_rotation','RHip_flexion','RHip_adduction', 
        'RKnee_flexion', 
        'RAnkle_adduction', 'RAnkle_flexion', 'RAnkle_rotation',
        'LHip_rotation','LHip_flexion','LHip_adduction', 
        'LKnee_flexion', 
        'LAnkle_adduction', 'LAnkle_flexion', 'LAnkle_rotation']

Theta = {}

for angle in Dofs:
    Theta[angle] = np.zeros(b2.shape[0])

achor_x = np.array([1, 0, 0])
achor_y = np.array([0, 1, 0])
achor_z = np.array([0, 0, 1])
achor_point = np.array([0, 0, 0])

SpineF = b2[:,:,3]
SpineM = b2[:,:,4]
SpineH = b2[:,:,5]
SpineM_f = SpineM
SpineM_f[:,2] = b2[:,2,3]
SpineM_h = SpineM
SpineM_h[:,2] = b2[:,2,5]

EarL = b2[:,:,0]
EarR = b2[:,:,1]
Snout = b2[:,:,2]
ForepawL = b2[:,:,8]
WristL = b2[:,:,9]
ElbowL = b2[:,:,10]
ShoulderL = b2[:,:,11]
ForepawR = b2[:,:,12]
WristR = b2[:,:,13]
ElbowR = b2[:,:,14]
ShoulderR = b2[:,:,15]
HindpawL = b2[:,:,16]
AnkleL = b2[:,:,17]
KneeL = b2[:,:,18]
HindpawR = b2[:,:,19]
AnkleR = b2[:,:,20]
KneeR = b2[:,:,21]

body = {
    'world': np.zeros_like(Snout),
    'head': Snout-SpineF,'LREar': EarR-EarL,
    'root': SpineF-SpineM,
    'RLShoulder': ShoulderL-ShoulderR,
    'RScapula': ShoulderR-SpineM,
    'RHumerus': ElbowR-ShoulderR,'RUlna': WristR-ElbowR, 'RRadius': WristR-ElbowR,'RFinger': ForepawR-WristR,
    'LScapula': ShoulderL-SpineM,
    'LHumerus': ElbowL-ShoulderL,'LUlna': WristL-ElbowL, 'LRadius': WristL-ElbowL,'LFinger': ForepawL-WristL,
    'Pelvis': SpineM-SpineH,
    'RFemur': KneeR-SpineH,'RLeg': AnkleR-KneeR,'RPedal': HindpawR-AnkleR,
    'LFemur': KneeL-SpineH,'LLeg': AnkleL-KneeL,'LPedal': HindpawL-AnkleL,
}

In [8]:
def project(A, B, C, D):
    # Calculate the normal vector of the plane
    BC = C - B
    BD = D - B
    normal = np.cross(BC, BD)

    # Find the equation of the plane
    plane_eq = np.dot(normal, B)

    # Determine the distance from A to the plane
    distance = np.dot(A, normal) - plane_eq

    # Project A onto the plane
    projected_A = A - distance * normal

    return projected_A

def angle(u, v):
    dot_product = np.dot(u, v)
    magnitude_u = np.linalg.norm(u)
    magnitude_v = np.linalg.norm(v)
    cosine_theta = dot_product / (magnitude_u * magnitude_v)
    angle_rad = np.arccos(np.clip(cosine_theta, -1.0, 1.0))  # Ensure the value is within valid range for arccos
    # angle_deg = np.degrees(angle_rad)
    return angle_rad

In [9]:
for i in range(b2.shape[0]):
    Theta[Dofs[0]][i], Theta[Dofs[1]][i], Theta[Dofs[2]][i] = (SpineF[i,:]+SpineM[i,:])/2

    # Theta[Dofs[3]][i] = angle(project(body['RLShoulder'][i], achor_point, achor_y, achor_z), achor_y) 
    Theta[Dofs[4]][i] = angle(project(body['root'][i], achor_point, achor_x, achor_z), achor_x) - 1.57
    Theta[Dofs[5]][i] = angle(project(body['root'][i], achor_point, achor_x, achor_y), achor_x)

    Theta[Dofs[6]][i] = angle(project(body['LREar'][i], ShoulderL[i,:], ShoulderR[i,:], (SpineF[i,:] + SpineM[i,:])/2), -1*body['RLShoulder'][i])
    Theta[Dofs[7]][i] = angle(project(body['head'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), (SpineF[i,:]-SpineM_f[i,:])) - np.pi/2
    Theta[Dofs[8]][i] = angle(project(body['head'][i], ShoulderL[i,:], ShoulderR[i,:], SpineF[i,:]), (SpineF[i,:]-SpineM_f[i,:])) - 1.57
    # Theta[Dofs[8]][i] = angle(project(body['head'][i], achor_point, achor_x, achor_y), project(body['root'][i], achor_point, achor_x, achor_y))

    Theta[Dofs[13]][i] = angle(project(body['RHumerus'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['root'][i]) - np.pi
    Theta[Dofs[14]][i] = angle(project(body['RHumerus'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['RHumerus'][i])
    Theta[Dofs[16]][i] = angle(body['RHumerus'][i], body['RUlna'][i])

    Theta[Dofs[18]][i] = angle(project(body['RFinger'][i], ShoulderR[i,:], ElbowR[i,:], WristR[i,:]), body['RFinger'][i]) 
    Theta[Dofs[19]][i] = angle(project(body['RFinger'][i], ShoulderR[i,:], ElbowR[i,:], WristR[i,:]), body['RUlna'][i]) - 1.57

    Theta[Dofs[26]][i] = angle(project(body['LHumerus'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['root'][i]) - np.pi
    Theta[Dofs[27]][i] = angle(project(body['LHumerus'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['LHumerus'][i])
    Theta[Dofs[29]][i] = angle(body['LHumerus'][i], body['LUlna'][i])

    Theta[Dofs[31]][i] = angle(project(body['LFinger'][i], ShoulderL[i,:], ElbowL[i,:], WristL[i,:]), body['LFinger'][i]) 
    Theta[Dofs[32]][i] = angle(project(body['LFinger'][i], ShoulderL[i,:], ElbowL[i,:], WristL[i,:]), body['LUlna'][i]) - 1.57

    Theta[Dofs[36]][i] = angle(project(body['Pelvis'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['root'][i]) - np.pi/2
    Theta[Dofs[37]][i] = angle(project(body['Pelvis'][i], SpineF[i,:], SpineM[i,:], SpineM_f[i,:]), body['Pelvis'][i])

    Theta[Dofs[39]][i] = angle(project(body['RFemur'][i], SpineH[i,:], SpineM[i,:], SpineM_h[i,:]), body['Pelvis'][i])
    Theta[Dofs[40]][i] = angle(project(body['RFemur'][i], SpineH[i,:], SpineM[i,:], SpineM_h[i,:]), body['RFemur'][i])
    Theta[Dofs[41]][i] = angle(body['RFemur'][i], body['RLeg'][i])

    Theta[Dofs[42]][i] = angle(project(body['RPedal'][i], SpineH[i,:], KneeR[i,:], AnkleR[i,:]), body['RPedal'][i]) 
    Theta[Dofs[43]][i] = angle(project(body['RPedal'][i], SpineH[i,:], KneeR[i,:], AnkleR[i,:]), body['RLeg'][i])

    Theta[Dofs[46]][i] = angle(project(body['LFemur'][i], SpineH[i,:], SpineM[i,:], SpineM_h[i,:]), body['Pelvis'][i])
    Theta[Dofs[47]][i] = angle(project(body['LFemur'][i], SpineH[i,:], SpineM[i,:], SpineM_h[i,:]), body['LFemur'][i])
    Theta[Dofs[48]][i] = angle(body['LFemur'][i], body['LLeg'][i])

    Theta[Dofs[49]][i] = angle(project(body['LPedal'][i], SpineH[i,:], KneeR[i,:], AnkleR[i,:]), body['LPedal'][i]) 
    Theta[Dofs[50]][i] = angle(project(body['LPedal'][i], SpineH[i,:], KneeR[i,:], AnkleR[i,:]), body['LLeg'][i])


In [10]:
Theta[Dofs[0]][:] -= np.min(Theta[Dofs[0]])
Theta[Dofs[1]][:] -= np.min(Theta[Dofs[1]])
Theta[Dofs[2]][:] -= np.min(Theta[Dofs[2]])

In [11]:
# Convert the dictionary values to a list of arrays
Theta_list = [v for v in Theta.values()]

# Concatenate the arrays along a new axis (axis=0)
qpos = np.concatenate([arr[:, np.newaxis] for arr in Theta_list], axis=1)

In [12]:
dt = mujoco_timestep

# Compute the change in joint angles
dqpos = np.diff(qpos, axis=0)

# Compute joint velocities
qvel = dqpos / dt

In [13]:
if np.isnan(qpos).any():
    print("qpos contains NaN values")
else:
    print("qpos does not contain NaN values")

# Check for NaN values in qvel
if np.isnan(qvel).any():
    print("qvel contains NaN values")
else:
    print("qvel does not contain NaN values")


qpos does not contain NaN values
qvel does not contain NaN values


In [14]:
directory_path = os.path.join('mocap_data', syllable_name)
os.makedirs(directory_path, exist_ok=True)
hdf5_file = os.path.join(directory_path,'data.h5')

with h5py.File(hdf5_file, 'w') as f:
    # Create datasets for qpos and qvel
    f.create_dataset('qpos', data=qpos[:-1,:])
    f.create_dataset('qvel', data=qvel)

print("Data written to HDF5 file successfully!")


Data written to HDF5 file successfully!


In [15]:
with h5py.File(hdf5_file, 'r') as f:
    print("Datasets in the HDF5 file:")
    for name in f:
        print(name, f[name].shape)

    # Check the contents of 'qpos' and 'qvel'
    qpos_data = f['qpos'][:]
    qvel_data = f['qvel'][:]
    print("Shape of 'qpos':", qpos_data.shape)
    print("Shape of 'qvel':", qvel_data.shape)

Datasets in the HDF5 file:
qpos (23, 52)
qvel (23, 52)
Shape of 'qpos': (23, 52)
Shape of 'qvel': (23, 52)


In [19]:
timestep_seconds = dt
trajectory_lengths = np.array([qvel.shape[0]])
directory_path = os.path.join('mocap_data', syllable_name)
os.makedirs(directory_path, exist_ok=True)
hdf5_file = os.path.join(directory_path, 'data_revised.h5')

with h5py.File(hdf5_file, 'w') as f:
    # Create id2name group
    id2name_grp = f.create_group('id2name')
    # Example datasets within id2name (fill with actual data as needed)
    id2name_grp.create_dataset('sites', data=np.array([b'site1', b'site2']))
    id2name_grp.create_dataset('joints', data=np.array([b'joint1', b'joint2']))
    id2name_grp.create_dataset('other', data=np.array([b'other1', b'other2']))

    # Create timestep_seconds dataset
    f.create_dataset('timestep_seconds', data=timestep_seconds)

    # Create trajectories group
    trajectories_grp = f.create_group('trajectories')
    
    # Create a subgroup for each trajectory
    for traj_idx in range(len(trajectory_lengths)):
        traj_grp = trajectories_grp.create_group(str(traj_idx).zfill(5))  # Example subgroup name
        # traj_grp.create_dataset('root_qpos', data=qpos[:-1,:])  # Example dataset, fill with actual root_qpos data
        traj_grp.create_dataset('qpos', data=qpos[:-1,:])
        # traj_grp.create_dataset('root_qvel', data=qvel)  # Example dataset, fill with actual root_qvel data
        traj_grp.create_dataset('qvel', data=qvel)
        # traj_grp.create_dataset('root2site', data=np.random.rand(23, 6))  # Example dataset, fill with actual data
        # traj_grp.create_dataset('joint_quat', data=np.random.rand(23, 8))  # Example dataset, fill with actual data

    # Create trajectory_lengths dataset
    f.create_dataset('trajectory_lengths', data=trajectory_lengths)

print("Data written to HDF5 file successfully!")

Data written to HDF5 file successfully!


In [3]:
filename_2 = r'D:\CyberMice\mocap_data\mocap_data\diving\data_revised.h5'

with h5py.File(filename_2, 'r') as f:
    # Get a list of top-level groups
    print("Top-level groups:", list(f.keys()))

    # Get a list of all groups and datasets
    for name, obj in f.items():
        print(name, obj)

f.close()

Top-level groups: ['id2name', 'timestep_seconds', 'trajectories', 'trajectory_lengths']
id2name <HDF5 group "/id2name" (3 members)>
timestep_seconds <HDF5 dataset "timestep_seconds": shape (), type "<f8">
trajectories <HDF5 group "/trajectories" (1 members)>
trajectory_lengths <HDF5 dataset "trajectory_lengths": shape (1,), type "<i4">
